# AlgoSpace Strategy - Optimized Production Version

**Version**: 4.0 (Optimized)
**Features**: 
- Ultra-fast Numba JIT compilation
- Efficient memory usage
- Robust error handling
- Production-ready code
- No emojis

In [ ]:
# === CELL 1: Environment Setup and Imports ===

import pandas as pd
import numpy as np
import vectorbt as vbt
import numba
from numba import jit, njit, prange, typed, types
from numba.experimental import jitclass
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

# Numba settings for maximum performance
numba.config.THREADING_LAYER = 'threadsafe'

print("AlgoSpace Multi-Indicator Trading Strategy")
print("Version: 4.0 - Optimized Production")
print("Status: Environment ready")

In [ ]:
# === CELL 2: Optimized Data Loading ===

def load_and_prepare_data(file_path, timeframe='5m'):
    """Optimized data loading with proper datetime handling"""
    # Read CSV with optimized settings
    df = pd.read_csv(file_path, parse_dates=True, infer_datetime_format=True)
    
    # Find datetime column
    datetime_cols = ['Timestamp', 'timestamp', 'Date', 'date', 'Time', 'time', 'Datetime', 'datetime']
    datetime_col = next((col for col in datetime_cols if col in df.columns), None)
    
    if datetime_col:
        # Parse datetime with multiple format attempts
        for date_format in ['%d/%m/%Y %H:%M', '%m/%d/%Y %H:%M', '%Y-%m-%d %H:%M:%S', None]:
            try:
                if date_format:
                    df['Datetime'] = pd.to_datetime(df[datetime_col], format=date_format)
                else:
                    df['Datetime'] = pd.to_datetime(df[datetime_col], dayfirst=True)
                
                if df['Datetime'].notna().sum() > len(df) * 0.8:
                    break
            except:
                continue
    
    # Set index
    if 'Datetime' in df.columns and df['Datetime'].notna().sum() > len(df) * 0.8:
        df.set_index('Datetime', inplace=True)
    else:
        # Create synthetic index
        freq = '5T' if timeframe == '5m' else '30T'
        df.index = pd.date_range(start='2020-01-01', periods=len(df), freq=freq)
    
    # Standardize columns
    column_map = {
        'open': 'Open', 'high': 'High', 'low': 'Low',
        'close': 'Close', 'volume': 'Volume'
    }
    df.rename(columns={k: v for k, v in column_map.items() if k in df.columns.str.lower()}, inplace=True)
    
    # Ensure numeric types
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Remove any NaN values
    df.dropna(subset=['Open', 'High', 'Low', 'Close'], inplace=True)
    
    return df

print("Data loading functions optimized")

In [ ]:
# === CELL 3: Load Data Files ===

# File paths
file_path_5min = "/home/QuantNova/AlgoSpace-Strategy-1/@NQ - 5 min - ETH.csv"
file_path_30min = "/home/QuantNova/AlgoSpace-Strategy-1/NQ - 30 min - ETH.csv"

# Load data with error handling
try:
    print("Loading 5-minute data...")
    df_5m = load_and_prepare_data(file_path_5min, '5m')
    print(f"5-minute data: {df_5m.shape[0]:,} rows, {df_5m.index[0]} to {df_5m.index[-1]}")
except Exception as e:
    print(f"Error loading 5-minute data: {e}")
    df_5m = pd.DataFrame()

try:
    print("\nLoading 30-minute data...")
    df_30m = load_and_prepare_data(file_path_30min, '30m')
    print(f"30-minute data: {df_30m.shape[0]:,} rows, {df_30m.index[0]} to {df_30m.index[-1]}")
except Exception as e:
    print(f"Error loading 30-minute data: {e}")
    df_30m = pd.DataFrame()

print("\nData loading complete")

In [ ]:
# === CELL 4: Ultra-Fast FVG Detection ===

@njit(parallel=True, fastmath=True, cache=True)
def detect_fvg_optimized(high, low, lookback=3, validity=20):
    """Ultra-fast FVG detection with parallel processing"""
    n = len(high)
    bull_fvg = np.zeros(n, dtype=np.bool_)
    bear_fvg = np.zeros(n, dtype=np.bool_)
    bull_active = np.zeros(n, dtype=np.bool_)
    bear_active = np.zeros(n, dtype=np.bool_)
    
    # Parallel FVG detection
    for i in prange(lookback, n):
        # Bullish FVG
        if low[i] > high[i-lookback]:
            bull_fvg[i] = True
            # Mark active period
            end_idx = min(i + validity, n)
            for j in range(i, end_idx):
                if low[j] >= high[i-lookback]:
                    bull_active[j] = True
                else:
                    break
        
        # Bearish FVG
        if high[i] < low[i-lookback]:
            bear_fvg[i] = True
            # Mark active period
            end_idx = min(i + validity, n)
            for j in range(i, end_idx):
                if high[j] <= low[i-lookback]:
                    bear_active[j] = True
                else:
                    break
    
    return bull_fvg, bear_fvg, bull_active, bear_active

# Calculate FVG
if not df_5m.empty:
    print("Calculating FVG indicators...")
    bull_fvg, bear_fvg, bull_active, bear_active = detect_fvg_optimized(
        df_5m['High'].values, df_5m['Low'].values
    )
    
    df_5m['FVG_Bull_Detected'] = bull_fvg
    df_5m['FVG_Bear_Detected'] = bear_fvg
    df_5m['FVG_Bull_Active'] = bull_active
    df_5m['FVG_Bear_Active'] = bear_active
    
    print(f"FVG Complete: Bull={bull_fvg.sum():,}, Bear={bear_fvg.sum():,}")

In [ ]:
# === CELL 5: Optimized MLMI with KNN ===

# MLMI Data Container
spec = [
    ('features', numba.float64[:, :]),
    ('labels', numba.int64[:]),
    ('size', numba.int64),
    ('capacity', numba.int64)
]

@jitclass(spec)
class MLMIData:
    def __init__(self, capacity):
        self.capacity = capacity
        self.features = np.zeros((capacity, 2), dtype=np.float64)
        self.labels = np.zeros(capacity, dtype=np.int64)
        self.size = 0
    
    def add(self, f1, f2, label):
        if self.size >= self.capacity:
            # Shift data
            shift = self.capacity // 4
            self.features[:-shift] = self.features[shift:]
            self.labels[:-shift] = self.labels[shift:]
            self.size = self.capacity - shift
        
        self.features[self.size, 0] = f1
        self.features[self.size, 1] = f2
        self.labels[self.size] = label
        self.size += 1

@njit(fastmath=True, cache=True)
def wma_fast(values, period):
    """Fast weighted moving average"""
    n = len(values)
    result = np.full(n, np.nan)
    
    if period > n:
        return result
    
    weights = np.arange(1, period + 1, dtype=np.float64)
    sum_weights = weights.sum()
    
    for i in range(period - 1, n):
        weighted_sum = 0.0
        for j in range(period):
            weighted_sum += values[i - period + j + 1] * weights[j]
        result[i] = weighted_sum / sum_weights
    
    return result

@njit(fastmath=True, cache=True)
def rsi_fast(prices, period):
    """Fast RSI calculation"""
    n = len(prices)
    rsi = np.full(n, 50.0)
    
    if period >= n:
        return rsi
    
    gains = np.zeros(n)
    losses = np.zeros(n)
    
    for i in range(1, n):
        diff = prices[i] - prices[i-1]
        if diff > 0:
            gains[i] = diff
        else:
            losses[i] = -diff
    
    avg_gain = np.mean(gains[1:period+1])
    avg_loss = np.mean(losses[1:period+1])
    
    if avg_loss > 0:
        rs = avg_gain / avg_loss
        rsi[period] = 100 - (100 / (1 + rs))
    else:
        rsi[period] = 100
    
    for i in range(period + 1, n):
        avg_gain = (avg_gain * (period - 1) + gains[i]) / period
        avg_loss = (avg_loss * (period - 1) + losses[i]) / period
        
        if avg_loss > 0:
            rs = avg_gain / avg_loss
            rsi[i] = 100 - (100 / (1 + rs))
        else:
            rsi[i] = 100
    
    return rsi

@njit(fastmath=True)
def knn_predict(features, labels, query, k):
    """Fast k-nearest neighbors prediction"""
    n = len(labels)
    if n == 0 or k == 0:
        return 0.0
    
    # Calculate distances
    distances = np.zeros(n)
    for i in range(n):
        dist = 0.0
        for j in range(2):
            diff = features[i, j] - query[j]
            dist += diff * diff
        distances[i] = np.sqrt(dist)
    
    # Find k nearest neighbors
    k = min(k, n)
    indices = np.argsort(distances)[:k]
    
    # Vote
    vote = 0
    for i in range(k):
        vote += labels[indices[i]]
    
    return float(vote)

def calculate_mlmi_fast(df, k_neighbors=200):
    """Fast MLMI calculation"""
    print("Calculating MLMI indicators...")
    
    close = df['Close'].values
    n = len(close)
    
    # Calculate indicators
    ma_fast = wma_fast(close, 5)
    ma_slow = wma_fast(close, 20)
    rsi_fast_vals = rsi_fast(close, 5)
    rsi_slow_vals = rsi_fast(close, 20)
    
    # Smooth RSI
    rsi_fast_smooth = wma_fast(rsi_fast_vals, 20)
    rsi_slow_smooth = wma_fast(rsi_slow_vals, 20)
    
    # Initialize MLMI data
    mlmi_data = MLMIData(min(10000, n))
    mlmi_values = np.zeros(n)
    
    # Process crossovers
    for i in range(1, n):
        # Detect crossovers
        if ma_fast[i] > ma_slow[i] and ma_fast[i-1] <= ma_slow[i-1]:
            if not np.isnan(rsi_fast_smooth[i]) and not np.isnan(rsi_slow_smooth[i]):
                label = 1 if i < n-1 and close[i+1] > close[i] else -1
                mlmi_data.add(rsi_slow_smooth[i], rsi_fast_smooth[i], label)
        
        elif ma_fast[i] < ma_slow[i] and ma_fast[i-1] >= ma_slow[i-1]:
            if not np.isnan(rsi_fast_smooth[i]) and not np.isnan(rsi_slow_smooth[i]):
                label = -1 if i < n-1 and close[i+1] < close[i] else 1
                mlmi_data.add(rsi_slow_smooth[i], rsi_fast_smooth[i], label)
        
        # Make prediction
        if mlmi_data.size > 0 and not np.isnan(rsi_fast_smooth[i]) and not np.isnan(rsi_slow_smooth[i]):
            query = np.array([rsi_slow_smooth[i], rsi_fast_smooth[i]])
            mlmi_values[i] = knn_predict(
                mlmi_data.features[:mlmi_data.size],
                mlmi_data.labels[:mlmi_data.size],
                query,
                min(k_neighbors, mlmi_data.size)
            )
    
    # Add to dataframe
    df['mlmi'] = mlmi_values
    df['mlmi_ma'] = wma_fast(mlmi_values, 20)
    df['mlmi_bull'] = mlmi_values > 0
    df['mlmi_bear'] = mlmi_values < 0
    
    # Crossovers
    mlmi_bull_cross = np.zeros(n, dtype=bool)
    mlmi_bear_cross = np.zeros(n, dtype=bool)
    
    for i in range(1, n):
        if mlmi_values[i] > 0 and mlmi_values[i-1] <= 0:
            mlmi_bull_cross[i] = True
        elif mlmi_values[i] < 0 and mlmi_values[i-1] >= 0:
            mlmi_bear_cross[i] = True
    
    df['mlmi_bull_cross'] = mlmi_bull_cross
    df['mlmi_bear_cross'] = mlmi_bear_cross
    
    print(f"MLMI Complete: Range=[{mlmi_values.min():.1f}, {mlmi_values.max():.1f}]")
    return df

# Calculate MLMI
if not df_30m.empty:
    df_30m = calculate_mlmi_fast(df_30m)

In [ ]:
# === CELL 6: Optimized NW-RQK ===

@njit(fastmath=True, cache=True)
def rational_quadratic_kernel(x, h, r):
    """Rational quadratic kernel function"""
    return (1.0 + (x * x) / (h * h * 2.0 * r)) ** (-r)

@njit(parallel=True, fastmath=True, cache=True)
def nadaraya_watson_fast(prices, h, r, min_periods=25):
    """Fast Nadaraya-Watson regression"""
    n = len(prices)
    result = np.full(n, np.nan)
    
    for i in prange(min_periods, n):
        weighted_sum = 0.0
        weight_sum = 0.0
        
        # Process window
        window_size = min(i + 1, 500)  # Limit window for performance
        
        for j in range(window_size):
            if i - j >= 0:
                weight = rational_quadratic_kernel(float(j), h, r)
                weighted_sum += prices[i - j] * weight
                weight_sum += weight
        
        if weight_sum > 0:
            result[i] = weighted_sum / weight_sum
    
    return result

def calculate_nwrqk_fast(df, h=8.0, r=8.0, lag=2):
    """Fast NW-RQK calculation"""
    print("Calculating NW-RQK indicators...")
    
    prices = df['Close'].values
    
    # Calculate regression lines
    yhat1 = nadaraya_watson_fast(prices, h, r)
    yhat2 = nadaraya_watson_fast(prices, h - lag, r)
    
    # Store in dataframe
    df['yhat1'] = yhat1
    df['yhat2'] = yhat2
    
    # Calculate signals
    n = len(df)
    bull_change = np.zeros(n, dtype=bool)
    bear_change = np.zeros(n, dtype=bool)
    bull_cross = np.zeros(n, dtype=bool)
    bear_cross = np.zeros(n, dtype=bool)
    
    for i in range(2, n):
        if not np.isnan(yhat1[i]) and not np.isnan(yhat1[i-1]) and not np.isnan(yhat1[i-2]):
            # Trend changes
            was_bear = yhat1[i-2] > yhat1[i-1]
            was_bull = yhat1[i-2] < yhat1[i-1]
            is_bear = yhat1[i-1] > yhat1[i]
            is_bull = yhat1[i-1] < yhat1[i]
            
            if is_bull and was_bear:
                bull_change[i] = True
            elif is_bear and was_bull:
                bear_change[i] = True
        
        # Crossovers
        if not np.isnan(yhat1[i]) and not np.isnan(yhat2[i]):
            if i > 0 and not np.isnan(yhat1[i-1]) and not np.isnan(yhat2[i-1]):
                if yhat2[i] > yhat1[i] and yhat2[i-1] <= yhat1[i-1]:
                    bull_cross[i] = True
                elif yhat2[i] < yhat1[i] and yhat2[i-1] >= yhat1[i-1]:
                    bear_cross[i] = True
    
    df['nwrqk_bull'] = bull_change
    df['nwrqk_bear'] = bear_change
    df['nwrqk_bull_cross'] = bull_cross
    df['nwrqk_bear_cross'] = bear_cross
    
    print(f"NW-RQK Complete: Bull={bull_change.sum():,}, Bear={bear_change.sum():,}")
    return df

# Calculate NW-RQK
if not df_30m.empty:
    df_30m = calculate_nwrqk_fast(df_30m)

In [ ]:
# === CELL 7: Data Alignment ===

def align_timeframes(df_30m, df_5m):
    """Efficiently align 30-minute indicators to 5-minute data"""
    print("\nAligning timeframes...")
    
    # Check datetime indices
    if not isinstance(df_5m.index, pd.DatetimeIndex) or not isinstance(df_30m.index, pd.DatetimeIndex):
        print("Error: Both dataframes must have datetime index")
        return pd.DataFrame()
    
    # Start with 5-minute data
    df = df_5m.copy()
    
    # Align 30-minute indicators
    indicators = ['mlmi', 'mlmi_bull', 'mlmi_bear', 'nwrqk_bull', 'nwrqk_bear']
    
    for indicator in indicators:
        if indicator in df_30m.columns:
            df[f'{indicator}_30m'] = df_30m[indicator].reindex(df.index, method='ffill')
    
    # Add derived features
    df['returns'] = df['Close'].pct_change()
    df['log_returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['volatility'] = df['returns'].rolling(20).std()
    df['atr'] = (df['High'] - df['Low']).rolling(20).mean()
    
    # Drop initial NaN rows
    df.dropna(inplace=True)
    
    print(f"Alignment complete: {len(df):,} rows")
    return df

# Prepare backtesting data
if not df_30m.empty and not df_5m.empty:
    df_backtest = align_timeframes(df_30m, df_5m)
    print(f"Backtest data ready: {df_backtest.index[0]} to {df_backtest.index[-1]}")
else:
    print("Missing required data")
    df_backtest = pd.DataFrame()

In [ ]:
# === CELL 8: Optimized Synergy Detection ===

@njit(fastmath=True, cache=True)
def detect_all_synergies(mlmi_bull, mlmi_bear, fvg_bull, fvg_bear, 
                        nwrqk_bull, nwrqk_bear, window=30):
    """Detect all 4 synergy types in one pass"""
    n = len(mlmi_bull)
    
    # Initialize signal arrays for all synergy types
    syn1_long = np.zeros(n, dtype=np.bool_)
    syn1_short = np.zeros(n, dtype=np.bool_)
    syn2_long = np.zeros(n, dtype=np.bool_)
    syn2_short = np.zeros(n, dtype=np.bool_)
    syn3_long = np.zeros(n, dtype=np.bool_)
    syn3_short = np.zeros(n, dtype=np.bool_)
    syn4_long = np.zeros(n, dtype=np.bool_)
    syn4_short = np.zeros(n, dtype=np.bool_)
    
    half_window = window // 2
    
    for i in range(2, n - window):
        # Type 1: MLMI → FVG → NWRQK
        if mlmi_bull[i] and not mlmi_bull[i-1]:
            for j in range(i+1, min(i+half_window, n)):
                if fvg_bull[j]:
                    for k in range(j+1, min(i+window, n)):
                        if nwrqk_bull[k]:
                            syn1_long[k] = True
                            break
                    break
        
        if mlmi_bear[i] and not mlmi_bear[i-1]:
            for j in range(i+1, min(i+half_window, n)):
                if fvg_bear[j]:
                    for k in range(j+1, min(i+window, n)):
                        if nwrqk_bear[k]:
                            syn1_short[k] = True
                            break
                    break
        
        # Type 2: MLMI → NWRQK → FVG
        if mlmi_bull[i] and not mlmi_bull[i-1]:
            for j in range(i+1, min(i+half_window, n)):
                if nwrqk_bull[j]:
                    for k in range(j+1, min(i+window, n)):
                        if fvg_bull[k]:
                            syn2_long[k] = True
                            break
                    break
        
        if mlmi_bear[i] and not mlmi_bear[i-1]:
            for j in range(i+1, min(i+half_window, n)):
                if nwrqk_bear[j]:
                    for k in range(j+1, min(i+window, n)):
                        if fvg_bear[k]:
                            syn2_short[k] = True
                            break
                    break
        
        # Type 3: NWRQK → MLMI → FVG
        if nwrqk_bull[i]:
            for j in range(i+1, min(i+half_window, n)):
                if mlmi_bull[j]:
                    for k in range(j+1, min(i+window, n)):
                        if fvg_bull[k]:
                            syn3_long[k] = True
                            break
                    break
        
        if nwrqk_bear[i]:
            for j in range(i+1, min(i+half_window, n)):
                if mlmi_bear[j]:
                    for k in range(j+1, min(i+window, n)):
                        if fvg_bear[k]:
                            syn3_short[k] = True
                            break
                    break
        
        # Type 4: NWRQK → FVG → MLMI
        if nwrqk_bull[i]:
            for j in range(i+1, min(i+half_window, n)):
                if fvg_bull[j]:
                    for k in range(j+1, min(i+window, n)):
                        if mlmi_bull[k]:
                            syn4_long[k] = True
                            break
                    break
        
        if nwrqk_bear[i]:
            for j in range(i+1, min(i+half_window, n)):
                if fvg_bear[j]:
                    for k in range(j+1, min(i+window, n)):
                        if mlmi_bear[k]:
                            syn4_short[k] = True
                            break
                    break
    
    return (syn1_long, syn1_short, syn2_long, syn2_short,
            syn3_long, syn3_short, syn4_long, syn4_short)

In [ ]:
# === CELL 9: Strategy Implementation ===

class OptimizedSynergyStrategy:
    """Optimized strategy implementation for all synergy types"""
    
    def __init__(self, data, synergy_type):
        self.data = data
        self.synergy_type = synergy_type
        self.synergy_names = {
            1: "Type 1: MLMI → FVG → NWRQK",
            2: "Type 2: MLMI → NWRQK → FVG",
            3: "Type 3: NWRQK → MLMI → FVG",
            4: "Type 4: NWRQK → FVG → MLMI"
        }
    
    def generate_signals(self):
        """Generate trading signals"""
        # Extract indicator arrays
        mlmi_bull = self.data['mlmi_bull_30m'].fillna(False).values
        mlmi_bear = self.data['mlmi_bear_30m'].fillna(False).values
        fvg_bull = self.data['FVG_Bull_Active'].values
        fvg_bear = self.data['FVG_Bear_Active'].values
        nwrqk_bull = self.data['nwrqk_bull_30m'].fillna(False).values
        nwrqk_bear = self.data['nwrqk_bear_30m'].fillna(False).values
        
        # Detect all synergies
        all_signals = detect_all_synergies(
            mlmi_bull, mlmi_bear, fvg_bull, fvg_bear, 
            nwrqk_bull, nwrqk_bear
        )
        
        # Select signals for this synergy type
        idx = (self.synergy_type - 1) * 2
        long_entries = all_signals[idx]
        short_entries = all_signals[idx + 1]
        
        # Simple exit logic
        n = len(self.data)
        long_exits = np.zeros(n, dtype=bool)
        short_exits = np.zeros(n, dtype=bool)
        
        return {
            'long_entries': pd.Series(long_entries, index=self.data.index),
            'short_entries': pd.Series(short_entries, index=self.data.index),
            'long_exits': pd.Series(long_exits, index=self.data.index),
            'short_exits': pd.Series(short_exits, index=self.data.index)
        }
    
    def backtest(self, init_cash=100000):
        """Run optimized backtest"""
        signals = self.generate_signals()
        
        print(f"\nBacktesting {self.synergy_names[self.synergy_type]}")
        print(f"Long signals: {signals['long_entries'].sum():,}")
        print(f"Short signals: {signals['short_entries'].sum():,}")
        
        if signals['long_entries'].sum() == 0 and signals['short_entries'].sum() == 0:
            print("No signals generated")
            return None
        
        try:
            # Create portfolio
            portfolio = vbt.Portfolio.from_signals(
                close=self.data['Close'],
                entries=signals['long_entries'] | signals['short_entries'],
                exits=signals['long_exits'] | signals['short_exits'],
                direction=np.where(signals['long_entries'], 1, 
                                 np.where(signals['short_entries'], -1, 0)),
                size=100,
                init_cash=init_cash,
                fees=0.0001,
                slippage=0.0001,
                freq='5T'
            )
            
            return portfolio
            
        except Exception as e:
            print(f"Backtest error: {e}")
            return None

In [ ]:
# === CELL 10: Run All Backtests ===

print("=" * 80)
print("RUNNING BACKTESTS FOR ALL SYNERGY TYPES")
print("=" * 80)

portfolios = {}
results = []

if not df_backtest.empty:
    for synergy_type in range(1, 5):
        strategy = OptimizedSynergyStrategy(df_backtest, synergy_type)
        portfolio = strategy.backtest()
        
        if portfolio:
            portfolios[synergy_type] = portfolio
            
            # Calculate metrics
            total_return = portfolio.total_return()
            sharpe_ratio = portfolio.sharpe_ratio()
            max_drawdown = portfolio.max_drawdown()
            total_trades = portfolio.stats()['Total Trades']
            win_rate = portfolio.stats()['Win Rate [%]']
            
            results.append({
                'Synergy': strategy.synergy_names[synergy_type],
                'Return': f"{total_return:.2%}",
                'Sharpe': f"{sharpe_ratio:.2f}",
                'Max DD': f"{max_drawdown:.2%}",
                'Trades': total_trades,
                'Win Rate': f"{win_rate:.1f}%"
            })
            
            print(f"\nResults for {strategy.synergy_names[synergy_type]}:")
            print(f"Total Return: {total_return:.2%}")
            print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
            print(f"Max Drawdown: {max_drawdown:.2%}")
            print(f"Total Trades: {total_trades:,}")
            print(f"Win Rate: {win_rate:.1f}%")
else:
    print("No data available for backtesting")

# Display summary table
if results:
    print("\n" + "=" * 80)
    print("SUMMARY TABLE")
    print("=" * 80)
    summary_df = pd.DataFrame(results)
    print(summary_df.to_string(index=False))

In [ ]:
# === CELL 11: Monte Carlo Validation ===

@njit(parallel=True, fastmath=True)
def monte_carlo_simulation(returns, n_sims=1000, n_periods=252):
    """Fast Monte Carlo simulation"""
    n_returns = len(returns)
    sim_returns = np.zeros(n_sims)
    sim_sharpes = np.zeros(n_sims)
    
    for i in prange(n_sims):
        # Random sampling with replacement
        indices = np.random.randint(0, n_returns, size=n_returns)
        sampled = returns[indices]
        
        # Calculate metrics
        total_return = np.prod(1 + sampled) - 1
        mean_return = np.mean(sampled)
        std_return = np.std(sampled)
        
        sim_returns[i] = total_return
        if std_return > 0:
            sim_sharpes[i] = mean_return / std_return * np.sqrt(n_periods)
        else:
            sim_sharpes[i] = 0
    
    return sim_returns, sim_sharpes

print("=" * 80)
print("MONTE CARLO VALIDATION")
print("=" * 80)

mc_results = []

for synergy_type, portfolio in portfolios.items():
    if portfolio:
        print(f"\nRunning Monte Carlo for Type {synergy_type}...")
        
        # Get returns
        returns = portfolio.returns().values
        returns = returns[~np.isnan(returns)]
        
        # Original metrics
        orig_return = portfolio.total_return()
        orig_sharpe = portfolio.sharpe_ratio()
        
        # Run simulation
        sim_returns, sim_sharpes = monte_carlo_simulation(returns)
        
        # Calculate percentiles
        return_pct = np.sum(sim_returns <= orig_return) / len(sim_returns) * 100
        sharpe_pct = np.sum(sim_sharpes <= orig_sharpe) / len(sim_sharpes) * 100
        
        # 95% confidence intervals
        return_ci_low = np.percentile(sim_returns, 2.5)
        return_ci_high = np.percentile(sim_returns, 97.5)
        sharpe_ci_low = np.percentile(sim_sharpes, 2.5)
        sharpe_ci_high = np.percentile(sim_sharpes, 97.5)
        
        mc_results.append({
            'Synergy': f"Type {synergy_type}",
            'Return Percentile': f"{return_pct:.1f}%",
            'Sharpe Percentile': f"{sharpe_pct:.1f}%",
            'Return CI': f"[{return_ci_low:.2%}, {return_ci_high:.2%}]",
            'Sharpe CI': f"[{sharpe_ci_low:.2f}, {sharpe_ci_high:.2f}]"
        })
        
        print(f"Return Percentile: {return_pct:.1f}%")
        print(f"Sharpe Percentile: {sharpe_pct:.1f}%")
        print(f"Average Percentile: {(return_pct + sharpe_pct) / 2:.1f}%")

# Display MC summary
if mc_results:
    print("\n" + "=" * 80)
    print("MONTE CARLO SUMMARY")
    print("=" * 80)
    mc_df = pd.DataFrame(mc_results)
    print(mc_df.to_string(index=False))

In [ ]:
# === CELL 12: Final Analysis and Recommendations ===

print("=" * 80)
print("FINAL ANALYSIS AND RECOMMENDATIONS")
print("=" * 80)

# Find best performers
if portfolios:
    best_return = -float('inf')
    best_sharpe = -float('inf')
    best_trades = 0
    
    best_return_type = None
    best_sharpe_type = None
    best_trades_type = None
    
    for synergy_type, portfolio in portfolios.items():
        if portfolio:
            ret = portfolio.total_return()
            sharpe = portfolio.sharpe_ratio()
            trades = portfolio.stats()['Total Trades']
            
            if ret > best_return:
                best_return = ret
                best_return_type = synergy_type
            
            if sharpe > best_sharpe:
                best_sharpe = sharpe
                best_sharpe_type = synergy_type
            
            if trades > best_trades:
                best_trades = trades
                best_trades_type = synergy_type
    
    print("\nPERFORMANCE LEADERS:")
    print(f"Best Return: Type {best_return_type} ({best_return:.2%})")
    print(f"Best Risk-Adjusted: Type {best_sharpe_type} (Sharpe: {best_sharpe:.2f})")
    print(f"Most Active: Type {best_trades_type} ({best_trades:,} trades)")
    
    print("\nKEY INSIGHTS:")
    print("1. Each synergy type captures unique market patterns")
    print("2. Consider combining multiple synergies for diversification")
    print("3. Monitor performance and adjust parameters regularly")
    print("4. Use position sizing based on confidence levels")
    
    print("\nRECOMMENDED APPROACH:")
    print("- Allocate capital across top 2-3 performing synergies")
    print("- Use Monte Carlo percentiles to size positions")
    print("- Implement proper risk management with stops")
    print("- Regular rebalancing based on performance")

print("\n" + "=" * 80)
print("STRATEGY ANALYSIS COMPLETE")
print("=" * 80)